In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import numpy as np
from keras.datasets import cifar10
from models.multiscale_vae import MultiscaleVariationalAutoencoder

import utils
import utils.transform

Using TensorFlow backend.


In [2]:
# run params
SECTION = "vae"
RUN_ID = "0002"
DATA_NAME = "digits"
BASE_DIR= "./run"
BASE_DIR_SECTION = "{0}/{1}/".format(BASE_DIR, SECTION)
RUN_FOLDER = BASE_DIR_SECTION + "_".join([RUN_ID, DATA_NAME])

if not os.path.exists(BASE_DIR):
    os.mkdir(BASE_DIR)
    
if not os.path.exists(BASE_DIR_SECTION):
    os.mkdir(BASE_DIR_SECTION)
    
if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, "viz"))
    os.mkdir(os.path.join(RUN_FOLDER, "images"))
    os.mkdir(os.path.join(RUN_FOLDER, "weights"))

mode = "build"

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [4]:
multiscale_vae = MultiscaleVariationalAutoencoder(
    input_dims=(32,32,3), 
    levels=4, 
    z_dims=[16,16,16,16])

In [5]:
multiscale_vae.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
scale_0 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
scale_1 (MaxPooling2D)          (None, 16, 16, 3)    0           scale_0[0][0]                    
__________________________________________________________________________________________________
scale_2 (MaxPooling2D)          (None, 8, 8, 3)      0           scale_1[0][0]                    
__________________________________________________________________________________________________
scale_3 (MaxPooling2D)          (None, 4, 4, 3)      0           scale_2[0][0]                    
____________________________________________________________________________________________

In [6]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10
KL_LOSS_FACTOR = 10

In [7]:
multiscale_vae.compile(
    learning_rate=LEARNING_RATE, 
    r_loss_factor=R_LOSS_FACTOR,
    kl_loss_factor=KL_LOSS_FACTOR
)

In [8]:
EPOCHS = 200
BATCH_SIZE = 16
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [9]:
# serialize model to JSON
model_json = multiscale_vae.model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
multiscale_vae.train(     
    x_train, 
    batch_size = BATCH_SIZE, 
    epochs = EPOCHS, 
    run_folder = RUN_FOLDER, 
    print_every_n_batches = PRINT_EVERY_N_BATCHES, 
    initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
 6896/50000 [===>..........................] - ETA: 3:24 - loss: 484.1941 - activation_1_loss: 110.8616 - add_1_loss: 112.0779 - add_2_loss: 122.8676 - add_3_loss: 138.3869 - activation_1_vae_r_loss: 1.1524 - activation_1_vae_kl_loss: 109.7091 - add_1_vae_r_loss: 2.3687 - add_1_vae_kl_loss: 109.7091 - add_2_vae_r_loss: 13.1585 - add_2_vae_kl_loss: 109.7091 - add_3_vae_r_loss: 28.6778 - add_3_vae_kl_loss: 109.7091   - ETA: 52:34 - loss: 1012.8100 - activation_1_loss: 242.3213 - add_1_loss: 243.6580 - add_2_loss: 254.8148 - add_3_loss: 272.0160 - activation_1_vae_r_loss: 1.1051 - activation_1_vae_kl_loss: 241.2162 - add_1_vae_r_loss: 2.4418 - add_1_vae_kl_loss: 241.2162 - add_2_vae_r_loss: 13.5987 -  - ETA: 7:57 - loss: 814.2867 - activation_1_loss: 192.9719 - add_1_loss: 194.2099 - add_2_loss: 205.1607 - add_3_loss: 221.9441 - activation_1_vae_r_loss: 1.1620 - activation_1_vae_kl_loss: 191.8099 - add_1_vae_r_loss: 2.4000 - add_1_vae_kl_loss: 191.8099 - add_2_vae_r_loss: 13.3